In [1]:
options(warn=-1)
library(dplyr)
library(DataExplorer)
library(ggplot2)
library(tidyr)
library(patchwork)
library(cowplot)
library(scales)
library(tidyverse)
library(reshape2)
library(kableExtra)
library(vcd)
library(performanceEstimation)
library(class)
library(ipred)
library(caret)
library(mlbench)
library(ROCR)
library(pROC)
library(MASS)
library(countrycode)
library(randomForest);


Dołączanie pakietu: ‘dplyr’


Następujące obiekty zostały zakryte z ‘package:stats’:

    filter, lag


Następujące obiekty zostały zakryte z ‘package:base’:

    intersect, setdiff, setequal, union



Dołączanie pakietu: ‘cowplot’


Następujący obiekt został zakryty z ‘package:patchwork’:

    align_plots


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.5
✔ lubridate 1.9.3     ✔ stringr   1.5.1
✔ purrr     1.0.2     ✔ tibble    3.2.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ readr::col_factor() masks scales::col_factor()
✖ purrr::discard()    masks scales::discard()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::lag()        masks stats::lag()
✖ lubridate::stamp()  masks cowplot::stamp()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Dołączanie pakietu: ‘reshape2’


Następujący obiekt został zakryty z ‘package:t

In [4]:
clean_data <- file.path("data", "clean_data.csv")
adult_data <- read.csv(file=clean_data, stringsAsFactors = TRUE, col.names=c("age", "workclass", "fnlwgt", "education", 
                                                       "marital.status", "occupation", "relationship", "race","sex",
                                                       "capital.gain", "capital.loss", "hours.per.week","native.country", "income"))

In [5]:
clean_test <- file.path("data", "clean_test.csv")
adult_test <- read.csv(file=clean_test, stringsAsFactors = TRUE, col.names=c("age", "workclass", "fnlwgt", "education", 
                                                       "marital.status", "occupation", "relationship", "race","sex",
                                                       "capital.gain", "capital.loss", "hours.per.week","native.country", "income"))

In [6]:
adult_data$income_binary <- as.integer(as.numeric(adult_data$income) - 1)
adult_test$income_binary <- as.integer(as.numeric(adult_test$income) - 1)

In [7]:
adult_data <- subset(adult_data, select = -c(income, fnlwgt))
adult_test <- subset(adult_test, select = -c(income, fnlwgt))

In [8]:
adult_data$capital <- as.integer(as.numeric(adult_data$capital.gain) - as.numeric(adult_data$capital.loss))
adult_test$capital <- as.integer(as.numeric(adult_test$capital.gain) - as.numeric(adult_test$capital.loss))

In [9]:
adult_data <- subset(adult_data, select = -c(capital.gain, capital.loss))
adult_test <- subset(adult_test, select = -c(capital.gain, capital.loss))

In [10]:
adult_data$continent <- countrycode(sourcevar = adult_data[, "native.country"],
                            origin = "country.name",
                            destination = "region")

In [11]:
country_continent_mapping <- c(England = 'Europe & Central Asia', Scotland = 'Europe & Central Asia', Columbia = 'South America', 
                               Hong = 'South Asia', South = 'South Asia')

In [12]:
adult_data$continent <- ifelse(is.na(adult_data$continent), 
                               country_continent_mapping[adult_data$native.country],
                               adult_data$continent)

In [13]:
adult_data$continent[adult_data$native.country == "England"] <- 'Europe & Central Asia'
adult_data$continent[adult_data$native.country == "Scotland"] <- 'Europe & Central Asia'
adult_data$continent[adult_data$native.country == "Hong"] <- 'South Asia'
adult_data$continent[adult_data$native.country == "South"] <- 'South Asia'

In [14]:
adult_data$continent <- as.factor(adult_data$continent)

In [15]:
str(adult_data)

'data.frame':	30161 obs. of  13 variables:
 $ age           : int  50 38 53 28 37 49 52 31 42 37 ...
 $ workclass     : Factor w/ 7 levels "Federal-gov",..: 5 3 3 3 3 3 5 3 3 3 ...
 $ education     : Factor w/ 16 levels "10th","11th",..: 10 12 2 10 13 7 12 13 10 16 ...
 $ marital.status: Factor w/ 7 levels "Divorced","Married-AF-spouse",..: 3 1 3 3 3 4 3 5 3 3 ...
 $ occupation    : Factor w/ 14 levels "Adm-clerical",..: 4 6 6 10 4 8 4 10 4 4 ...
 $ relationship  : Factor w/ 6 levels "Husband","Not-in-family",..: 1 2 1 6 6 2 1 2 1 1 ...
 $ race          : Factor w/ 5 levels "Amer-Indian-Eskimo",..: 5 5 3 3 5 3 5 5 5 3 ...
 $ sex           : Factor w/ 2 levels "Female","Male": 2 2 2 1 1 1 2 1 2 2 ...
 $ hours.per.week: int  13 40 40 40 40 16 45 50 40 80 ...
 $ native.country: Factor w/ 41 levels "Cambodia","Canada",..: 39 39 39 5 39 23 39 39 39 39 ...
 $ income_binary : int  0 0 0 0 0 0 1 1 1 1 ...
 $ capital       : int  0 0 0 0 0 0 0 14084 5178 0 ...
 $ continent     : Factor w/ 6 lev

In [16]:
adult_test$continent <- countrycode(sourcevar = adult_test[, "native.country"],
                            origin = "country.name",
                            destination = "region")

In [17]:
adult_test$continent <- ifelse(is.na(adult_test$continent), 
                               country_continent_mapping[adult_test$native.country],
                               adult_test$continent)

In [18]:
adult_test$continent[adult_test$native.country == "England"] <- 'Europe & Central Asia'
adult_test$continent[adult_test$native.country == "Scotland"] <- 'Europe & Central Asia'
adult_test$continent[adult_test$native.country == "Hong"] <- 'South Asia'
adult_test$continent[adult_test$native.country == "South"] <- 'South Asia'

In [19]:
adult_test$continent <- as.factor(adult_test$continent)

In [20]:
adult_data <- subset(adult_data, select = -c(native.country))
adult_test <- subset(adult_test, select = -c(native.country))

In [22]:
(adult_data %>% sample_n(5))

age,workclass,education,marital.status,occupation,relationship,race,sex,hours.per.week,income_binary,capital,continent
<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<fct>
21,Private,HS-grad,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,32,0,0,North America
32,Local-gov,Bachelors,Married-civ-spouse,Prof-specialty,Wife,White,Female,45,0,5013,North America
28,Private,HS-grad,Never-married,Other-service,Not-in-family,Black,Male,40,0,0,North America
35,Private,HS-grad,Never-married,Handlers-cleaners,Not-in-family,White,Male,40,0,0,North America
46,Private,Some-college,Married-civ-spouse,Sales,Husband,White,Male,40,0,0,North America


In [23]:
(adult_test %>% sample_n(5))

age,workclass,education,marital.status,occupation,relationship,race,sex,hours.per.week,income_binary,capital,continent
<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<fct>
20,Private,Some-college,Married-civ-spouse,Sales,Husband,White,Male,40,0,0,North America
36,Private,Some-college,Married-civ-spouse,Craft-repair,Husband,White,Male,40,1,0,North America
50,Self-emp-inc,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,Male,45,1,7298,North America
45,Self-emp-not-inc,HS-grad,Married-civ-spouse,Craft-repair,Husband,Asian-Pac-Islander,Male,45,0,0,South Asia
40,Private,Some-college,Never-married,Machine-op-inspct,Not-in-family,Black,Female,40,0,0,North America


In [24]:
write.csv(adult_data, file = "data/train_data.csv", row.names = FALSE)

In [25]:
write.csv(adult_test, file = "data/test_data.csv", row.names = FALSE)

In [26]:
train_data <- read.csv('data/train_data.csv', stringsAsFactors = TRUE)

In [27]:
(train_data %>% sample_n(5))

age,workclass,education,marital.status,occupation,relationship,race,sex,hours.per.week,income_binary,capital,continent
<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<fct>
23,Private,Some-college,Never-married,Craft-repair,Not-in-family,White,Male,40,0,0,North America
43,Private,HS-grad,Divorced,Machine-op-inspct,Own-child,White,Female,40,0,0,Latin America & Caribbean
18,Private,Some-college,Never-married,Other-service,Own-child,White,Male,20,0,0,North America
43,Private,Prof-school,Never-married,Prof-specialty,Not-in-family,White,Male,66,0,0,Europe & Central Asia
57,Private,12th,Married-civ-spouse,Transport-moving,Husband,Black,Male,40,1,7688,North America


In [28]:
test_data <- read.csv('data/test_data.csv', stringsAsFactors = TRUE)

In [29]:
(test_data %>% sample_n(5))

age,workclass,education,marital.status,occupation,relationship,race,sex,hours.per.week,income_binary,capital,continent
<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<fct>
38,Federal-gov,Some-college,Never-married,Adm-clerical,Not-in-family,White,Female,40,0,-1980,North America
35,Private,Bachelors,Divorced,Other-service,Own-child,White,Female,40,0,0,North America
31,Private,Some-college,Never-married,Other-service,Not-in-family,White,Female,20,0,0,North America
39,Private,Some-college,Married-civ-spouse,Adm-clerical,Husband,White,Male,40,1,0,North America
44,Local-gov,Bachelors,Never-married,Prof-specialty,Not-in-family,White,Male,55,0,0,North America


In [30]:
X_train <- subset(train_data, select = -c(income_binary))
X_test <- subset(test_data, select = -c(income_binary))

In [32]:
(X_train %>% sample_n(5))

age,workclass,education,marital.status,occupation,relationship,race,sex,hours.per.week,capital,continent
<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<fct>
44,Private,Bachelors,Married-civ-spouse,Transport-moving,Husband,White,Male,55,0,North America
28,State-gov,Bachelors,Never-married,Prof-specialty,Own-child,White,Male,40,6849,North America
20,Private,Some-college,Never-married,Other-service,Own-child,White,Male,40,0,North America
34,Federal-gov,Bachelors,Never-married,Exec-managerial,Unmarried,Asian-Pac-Islander,Male,40,1471,East Asia & Pacific
26,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,40,0,North America


In [33]:
(X_test %>% sample_n(5))

age,workclass,education,marital.status,occupation,relationship,race,sex,hours.per.week,capital,continent
<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<fct>
66,Self-emp-not-inc,HS-grad,Widowed,Tech-support,Not-in-family,White,Female,45,0,North America
35,Federal-gov,HS-grad,Married-civ-spouse,Other-service,Wife,White,Female,40,7298,North America
42,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,40,0,North America
39,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,40,0,North America
35,Private,HS-grad,Married-civ-spouse,Tech-support,Wife,White,Female,35,0,North America


In [31]:
Y_train <- subset(train_data, select = c(income_binary))
Y_test <- subset(test_data, select = c(income_binary))

In [37]:
X = rbind(X_train, X_test)
Y = rbind(Y_train, Y_test)

In [39]:
numerical_columns <- names(X[sapply(X, is.numeric)])
factor_columns <- names(clean_data[sapply(clean_data, is.factor)])

ERROR: Error in colMeans(x, na.rm = TRUE): argument 'x' musi być liczbą
